# Convolutional Neural Network

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets,transforms


In [2]:
input_size=28 # Figs size 28*28
num_classes=10 # Numbers of labels
num_epochs=3 # Training iteration
batch_size=64 # 64 Figs

# Dataset from MNIST
train_dataset=datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(
                in_channels=1,          # Gray Scale Image
                out_channels=16,        # Output
                kernel_size=5,          # Kernel size
                stride=1,               # step
                padding=2,              # Added side
            ),                          # Output (16,28,28)
            nn.ReLU() ,                 # ReLU
            nn.MaxPool2d(kernel_size=2),# Max Pool output (16,14,14)
        )
        self.conv2=nn.Sequential(       # Input(16,14,14)
            nn.Conv2d(
                in_channels=16,         # Gray Scale Image
                out_channels=32,        # Output
                kernel_size=5,          # Kernel size
                stride=1,               # step
                padding=2,              # Added side
            ),                          # Output (32,14,14)
            nn.ReLU(),                  # ReLU
            nn.MaxPool2d(kernel_size=2),# Max Pool output (32,7,7)
        )
        self.out=nn.Linear(in_features=32*7*7,out_features=10) # Fully connected layers

    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=x.view(x.size(0),-1)          # Flatten
        output = self.out(x)
        return output

In [4]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data,1)[1]
    rights =pred.eq(labels.data.view_as(pred)).sum()
    return rights,len(labels)

In [5]:
net = CNN()

criterion=nn.CrossEntropyLoss()

optimizer=optim.Adam(net.parameters(),lr=0.001)

for epoch in range(num_epochs):
    train_rights=[]

    for batch_idx,(data,target) in enumerate(train_loader): # Iterration for each batch
        net.train()
        output=net(data)
        loss=criterion(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right=accuracy(output,target)
        train_rights.append(right)

        if batch_idx % 100==0:
            net.eval()
            val_rights=[]
            for (data,target) in test_loader:
                output=net(data)
                right=accuracy(output,target)
                val_rights.append(right)
            # Caculate accuracy score
            train_r=(sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            val_r=(sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
            print('epoch: {} [{}/{}({:.0f}%)]\tloss: {:.6f}\tTrain set accuracy: {:.2f}%\tTest set accuracy: {:.2f}%'.format(
            epoch,batch_idx*batch_size,len(train_loader.dataset),100.*batch_idx/len(train_loader),loss.data,100.*train_r[0].numpy()/train_r[1],100.*val_r[0].numpy()/val_r[1]
            ))


epoch: 0 [0/60000(0%)]	loss: 2.318325	Train set accuracy: 7.81%	Test set accuracy: 15.91%
epoch: 0 [6400/60000(11%)]	loss: 0.425031	Train set accuracy: 72.88%	Test set accuracy: 92.19%
epoch: 0 [12800/60000(21%)]	loss: 0.089556	Train set accuracy: 83.22%	Test set accuracy: 95.39%
epoch: 0 [19200/60000(32%)]	loss: 0.067831	Train set accuracy: 87.29%	Test set accuracy: 96.33%
epoch: 0 [25600/60000(43%)]	loss: 0.035491	Train set accuracy: 89.59%	Test set accuracy: 96.10%
epoch: 0 [32000/60000(53%)]	loss: 0.021460	Train set accuracy: 91.05%	Test set accuracy: 97.53%
epoch: 0 [38400/60000(64%)]	loss: 0.077475	Train set accuracy: 92.10%	Test set accuracy: 97.73%
epoch: 0 [44800/60000(75%)]	loss: 0.038107	Train set accuracy: 92.85%	Test set accuracy: 97.62%
epoch: 0 [51200/60000(85%)]	loss: 0.077965	Train set accuracy: 93.43%	Test set accuracy: 98.08%
epoch: 0 [57600/60000(96%)]	loss: 0.015774	Train set accuracy: 93.92%	Test set accuracy: 97.93%
epoch: 1 [0/60000(0%)]	loss: 0.047839	Train set